# 1번 방법 : 블로그에 준비해둔 데이터를 url통해서 가져오기.
## 문제점1 : 블로그는 javascript 기반이라 selenium을 이용해야 한다고 합니다.
## 문제점2 : selenium 을 사용한다고 해도 데이터 테이블 자체가 tr, td 로 가지런히 배열된 느낌이 아니에요 초콜릿 판 처럼 독립된 틀이 많아서 이것도 제가 selector 지정을 잘 이해 못하는 문제 같아요

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

# ChromeDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://blog.naver.com/hellocube/223615712262'
driver.get(url)

# 페이지가 로드될 시간을 주기 위해 잠시 대기
time.sleep(3)  # 필요한 경우 대기 시간 조정

# CSV 파일로 저장
with open('table_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 특정 ID를 가진 span 요소들 선택
    span_elements = driver.find_elements(By.CSS_SELECTOR, 'span[id^="SE-"]')  # "SE-"로 시작하는 모든 span 요소 선택
    
    for element in span_elements:
        value = element.text.strip()  # 값 추출
        writer.writerow([value])  # CSV에 값 저장

# 브라우저 닫기
driver.quit()

print("CSV 파일로 저장되었습니다.")

# 2번 방법 : 교통정보 사이트에서 url 찾기
## 문제점 : url 특정이 어려운 상태. F12누른 후, 네트워크 창에서 url 정보를 찾을 수 있다고 하는데 잘 못찾겠어요
### : url만 찾을 수 있다면 selector 찾은 후 코드 작성이 가능할 듯 합니다.

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# URL 설정
url = 'https://taas.koroad.or.kr/sta/acs/exs/selectPatternInfo.do?subjectId=138&patternId=149&termsYn=Y&_ts=1012%7C0719%7C774'

# 웹 페이지 요청
res = requests.get(url)

# 응답 페이지에서 필요한 정보 추출
if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # 표 데이터 추출
    table = soup.find('table')
    table_rows = table.select('tbody tr')  # tbody 내의 모든 행 선택

    # CSV 파일로 저장
    with open('table_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        for row in table_rows:
            cells = row.select('td')  # 데이터 셀 선택
            cell_data = [cell.get_text(strip=True) for cell in cells]  # 텍스트 추출
            writer.writerow(cell_data)

    print("CSV 파일로 저장되었습니다.")
else:
    print(f"요청 실패. 응답 코드: {res.status_code}")


AttributeError: 'NoneType' object has no attribute 'select'

In [ ]:
# 연료 유형별 고속도로 연비 값 추가
for item in all_data:
    fuel = item.get('연료')
    if fuel in fuel_displacement and item.get('도심주행연비') is not None:
        try:
            # 고속도로 연비 값을 float로 변환
            fuel_displacement[fuel].append(float(item['도심주행연비']))
        except (ValueError, TypeError):
            continue  # 변환할 수 없는 경우 무시

# 평균 계산 및 결과 출력
for fuel, values in fuel_displacement.items():
    if values:
        average_displacement = sum(values) / len(values)
    else:
        average_displacement = 0
    print(f"{fuel} 차량의 도심주행연비: {values}")
    print(f"{fuel} 차량의 평균 도심주행연비: {average_displacement:.2f}")


# 3번 방법 : 외부 통계표를 특정 url로 찾았습니다 현재 방법으로는 가장 가능성이 있어보여요
### span class = 'val' 인 값으로 찾으면 좋을 것 같아요
### 현재 selector는 #mainTable > tbody > tr:nth-child(9) > td:nth-child(3) > span.val 중에 tr 부분까지 라고 생각합니다.


In [5]:
import requests
from bs4 import BeautifulSoup
import csv

# URL 설정
url = 'https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2'

# 웹 페이지 요청
response = requests.get(url)

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# span class='val' 요소 찾기
values = soup.find_all('span', class_='val')

# CSV 파일로 저장
with open('data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Value'])  # 헤더 작성
    
    for value in values:
        writer.writerow([value.text])  # 각 값을 한 행에 작성

print("Data has been saved to data.csv")


Data has been saved to data.csv


In [1]:
import requests
from bs4 import BeautifulSoup

# 대상 URL 설정
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"

# requests를 사용하여 웹 페이지 가져오기
response = requests.get(url)

# 페이지 가져오기 성공 여부 확인
if response.status_code == 200:
    # BeautifulSoup 객체 생성 (html.parser로 파싱)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 'span.val'로 끝나는 모든 요소 찾기
    values = soup.select('span.val')

    # 결과 출력
    for value in values:
        print(value.text.strip())  # 값에서 공백 제거 후 출력

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


In [2]:
import requests
from bs4 import BeautifulSoup
import csv

# 대상 URL 설정
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"

# requests를 사용하여 웹 페이지 가져오기
response = requests.get(url)

# 페이지 가져오기 성공 여부 확인
if response.status_code == 200:
    # BeautifulSoup 객체 생성 (html.parser로 파싱)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 'span.val'로 끝나는 모든 요소 찾기
    values = soup.select('span.val')

    # 크롤링한 데이터를 저장할 리스트
    data = []

    # 각 값 추출 및 리스트에 추가
    for value in values:
        data.append([value.text.strip()])  # 각 값을 리스트로 감싸서 추가

    # CSV 파일로 저장
    with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Value'])  # 헤더 작성
        writer.writerows(data)  # 데이터 작성

    print("Data successfully written to output_data.csv")

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Data successfully written to output_data.csv


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 'span.val' 선택자에 해당하는 모든 요소 찾기
values = driver.find_elements(By.CSS_SELECTOR, 'span.val')

# 크롤링한 데이터를 저장할 리스트
data = []

# 각 요소의 텍스트 값을 추출하여 리스트에 저장
for value in values:
    data.append([value.text.strip()])

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Value'])  # 헤더 작성
    writer.writerows(data)  # 데이터 작성

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 'span.val' 선택자에 해당하는 모든 요소 찾기
values_val = driver.find_elements(By.CSS_SELECTOR, 'span.val')

# 'span:nth-child(3)' 선택자에 해당하는 모든 요소 찾기
values_nth_child = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')

# 크롤링한 데이터를 저장할 리스트
data = []

# 'span.val'에서 텍스트 값을 추출하여 리스트에 저장
for value in values_val:
    data.append([value.text.strip()])

# 'span:nth-child(3)'에서 텍스트 값을 추출하여 리스트에 저장
for value in values_nth_child:
    data.append([value.text.strip()])

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Value'])  # 헤더 작성
    writer.writerows(data)  # 데이터 작성

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 'span:nth-child(3)' 선택자에 해당하는 첫 번째 열의 데이터 찾기
first_column_values = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')

# 두 번째 열부터 71번째 열까지 'td:nth-child(X) > span.val' 선택자에 해당하는 데이터 찾기
second_and_more_columns_values = []
for i in range(3, 72):  # 3부터 71까지 반복
    selector = f'td:nth-child({i}) > span.val'
    column_values = driver.find_elements(By.CSS_SELECTOR, selector)
    second_and_more_columns_values.append([value.text.strip() for value in column_values])

# CSV 파일로 저장할 데이터 준비
data = []

# 각 열의 데이터를 병합하여 저장
for idx, value in enumerate(first_column_values):
    row = [value.text.strip()]  # 첫 번째 열의 값 추가
    for col_values in second_and_more_columns_values:
        # 인덱스에 해당하는 값을 추가 (열의 수가 맞는지 확인)
        row.append(col_values[idx] if idx < len(col_values) else "")
    data.append(row)

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV 헤더 작성
    header = ['First Column'] + [f'Column {i}' for i in range(3, 72)]
    writer.writerow(header)
    
    # 데이터 작성
    writer.writerows(data)

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 'span:nth-child(3)' 선택자에 해당하는 첫 번째 열의 데이터 찾기
first_column_values = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')

# 다섯 번째 데이터부터 추출 (인덱스 4부터 시작)
first_column_values = first_column_values[4:]

# 두 번째 열부터 71번째 열까지 'td:nth-child(X) > span.val' 선택자에 해당하는 데이터 찾기
second_and_more_columns_values = []
for i in range(3, 72):  # 3부터 71까지 반복
    selector = f'td:nth-child({i}) > span.val'
    column_values = driver.find_elements(By.CSS_SELECTOR, selector)
    # 두 번째 열부터 71번째 열의 값들도 다섯 번째 데이터부터 사용
    second_and_more_columns_values.append([value.text.strip() for value in column_values[4:]])

# CSV 파일로 저장할 데이터 준비
data = []

# 각 열의 데이터를 병합하여 저장
for idx, value in enumerate(first_column_values):
    row = [value.text.strip()]  # 첫 번째 열의 값 추가
    for col_values in second_and_more_columns_values:
        # 인덱스에 해당하는 값을 추가 (열의 수가 맞는지 확인)
        row.append(col_values[idx] if idx < len(col_values) else "")
    data.append(row)

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV 헤더 작성
    header = ['First Column'] + [f'Column {i}' for i in range(3, 72)]
    writer.writerow(header)
    
    # 데이터 작성
    writer.writerows(data)

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 'span:nth-child(3)' 선택자에 해당하는 첫 번째 열의 데이터 찾기
first_column_values = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')

# 다섯 번째 데이터부터 추출 (인덱스 4부터 시작)
first_column_values = first_column_values[4:]

# 두 번째 열부터 71번째 열까지 'td:nth-child(X) > span.val' 선택자에 해당하는 데이터 찾기
second_and_more_columns_values = []
for i in range(3, 72):  # 3부터 71까지 반복
    selector = f'td:nth-child({i}) > span.val'
    column_values = driver.find_elements(By.CSS_SELECTOR, selector)
    # 열 데이터 모두 추가 (처음부터)
    second_and_more_columns_values.append([value.text.strip() for value in column_values])

# CSV 파일로 저장할 데이터 준비
data = []

# 각 열의 데이터를 병합하여 저장
for idx, value in enumerate(first_column_values):
    row = [value.text.strip()]  # 첫 번째 열의 값 추가 (다섯 번째부터 시작)
    for col_values in second_and_more_columns_values:
        # 두 번째 열부터 71번째 열의 값은 처음부터 모두 추가
        row.append(col_values[idx] if idx < len(col_values) else "")
    data.append(row)

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV 헤더 작성
    header = ['First Column'] + [f'Column {i}' for i in range(3, 72)]
    writer.writerow(header)
    
    # 데이터 작성
    writer.writerows(data)

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 'span:nth-child(3)' 선택자에 해당하는 첫 번째 열의 데이터 찾기 (다섯 번째 데이터부터)
first_column_values = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')
first_column_values = first_column_values[4:]

# 'span:nth-child(2)' 선택자로 두 번째 열부터 컬럼명 가져오기
column_names_elements = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(2)')

# 두 번째 열부터 71번째 열까지 'td:nth-child(X) > span.val' 선택자에 해당하는 데이터 찾기
second_and_more_columns_values = []
for i in range(3, 72):  # 3부터 71까지 반복
    selector = f'td:nth-child({i}) > span.val'
    column_values = driver.find_elements(By.CSS_SELECTOR, selector)
    # 열 데이터 모두 추가 (처음부터)
    second_and_more_columns_values.append([value.text.strip() for value in column_values])

# CSV 파일로 저장할 데이터 준비
data = []

# 각 열의 데이터를 병합하여 저장
for idx, value in enumerate(first_column_values):
    row = [value.text.strip()]  # 첫 번째 열의 값 추가 (다섯 번째부터 시작)
    for col_values in second_and_more_columns_values:
        # 두 번째 열부터 71번째 열의 값은 처음부터 모두 추가
        row.append(col_values[idx] if idx < len(col_values) else "")
    data.append(row)

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV 헤더 작성 (첫 번째 열은 '자치구별', 나머지는 'span:nth-child(2)'에서 추출한 값)
    header = ['자치구별'] + [name.text.strip() for name in column_names_elements[:69]]  # 두 번째부터 71번째 열의 이름 추가
    writer.writerow(header)
    
    # 데이터 작성
    writer.writerows(data)

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'  # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 첫 번째 열 '자치구별' 명명
column_names = ['자치구별']

# 두 번째 열부터 큰 카테고리 가져오기 (tr:nth-child(3))
large_category_elements = []
for i in range(1, 19):  # th:nth-child(1)부터 th:nth-child(18)까지
    selector = f'tr:nth-child(3) > th:nth-child({i}) > span:nth-child(2)'
    element = driver.find_element(By.CSS_SELECTOR, selector)
    large_category_elements.append(element.text.strip())

# 세부 칼럼명 가져오기 (tr:nth-child(4))
detailed_column_elements = []
for i in range(1, 70):  # th:nth-child(1)부터 th:nth-child(69)까지
    selector = f'tr:nth-child(4) > th:nth-child({i}) > span:nth-child(2)'
    element = driver.find_element(By.CSS_SELECTOR, selector)
    detailed_column_elements.append(element.text.strip())

# 큰 카테고리에 따른 세부 칼럼명 설정
for idx, large_category in enumerate(large_category_elements):
    if large_category == "이륜차":
        # '이륜차'는 3개의 열을 묶음
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 3]}")
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 3 + 1]}")
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 3 + 2]}")
    else:
        # 그 외는 4개의 열을 묶음
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 4]}")
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 4 + 1]}")
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 4 + 2]}")
        column_names.append(f"{large_category}_{detailed_column_elements[idx * 4 + 3]}")

# 데이터 크롤링 (span:nth-child(3) -> 자치구별, td:nth-child(3) > span.val -> 나머지 열 데이터)
first_column_values = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')
first_column_values = first_column_values[4:]  # 첫 번째 열의 값 (다섯 번째부터 시작)

# 두 번째 열부터 71번째 열까지의 데이터 찾기
second_and_more_columns_values = []
for i in range(3, 72):  # 3부터 71까지 반복
    selector = f'td:nth-child({i}) > span.val'
    column_values = driver.find_elements(By.CSS_SELECTOR, selector)
    # 열 데이터 모두 추가 (처음부터)
    second_and_more_columns_values.append([value.text.strip() for value in column_values])

# CSV 파일로 저장할 데이터 준비
data = []

# 각 열의 데이터를 병합하여 저장
for idx, value in enumerate(first_column_values):
    row = [value.text.strip()]  # 첫 번째 열의 값 추가 (다섯 번째부터 시작)
    for col_values in second_and_more_columns_values:
        # 두 번째 열부터 71번째 열의 값은 처음부터 모두 추가
        row.append(col_values[idx] if idx < len(col_values) else "")
    data.append(row)

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV 헤더 작성
    writer.writerow(column_names)
    
    # 데이터 작성
    writer.writerows(data)

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

# ChromeDriver 경로 설정
chrome_driver_path = r'C:\Users\Playdata\Documents\chromedriver-win64\chromedriver.exe'   # ChromeDriver 경로 수정

# Service 객체 생성
service = Service(executable_path=chrome_driver_path)

# 웹 드라이버 실행 (크롬)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = "https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기 (필요에 따라 시간 조정)
time.sleep(5)

# 첫 번째 열 '자치구별' 명명
column_names = ['자치구별']

# 두 번째 열부터 큰 카테고리 가져오기 (tr:nth-child(3))
large_category_elements = []
for i in range(1, 19):  # th:nth-child(1)부터 th:nth-child(18)까지
    selector = f'tr:nth-child(3) > th:nth-child({i}) > span:nth-child(2)'
    element = driver.find_element(By.CSS_SELECTOR, selector)
    large_category_elements.append(element.text.strip())

# 큰 카테고리에 따른 세부 칼럼명 설정
for large_category in large_category_elements:
    if large_category == "이륜차":
        # '이륜차'는 3개의 열을 묶음 ('계', '관용', '자가용')
        column_names.append(f"{large_category}_계")
        column_names.append(f"{large_category}_관용")
        column_names.append(f"{large_category}_자가용")
    else:
        # 그 외는 4개의 열을 묶음 ('계', '관용', '자가용', '영업용')
        column_names.append(f"{large_category}_계")
        column_names.append(f"{large_category}_관용")
        column_names.append(f"{large_category}_자가용")
        column_names.append(f"{large_category}_영업용")

# 데이터 크롤링 (span:nth-child(3) -> 자치구별, td:nth-child(3) > span.val -> 나머지 열 데이터)
first_column_values = driver.find_elements(By.CSS_SELECTOR, 'span:nth-child(3)')
first_column_values = first_column_values[4:]  # 첫 번째 열의 값 (다섯 번째부터 시작)

# 두 번째 열부터 71번째 열까지의 데이터 찾기
second_and_more_columns_values = []
for i in range(3, 72):  # 3부터 71까지 반복
    selector = f'td:nth-child({i}) > span.val'
    column_values = driver.find_elements(By.CSS_SELECTOR, selector)
    # 열 데이터 모두 추가 (처음부터)
    second_and_more_columns_values.append([value.text.strip() for value in column_values])

# CSV 파일로 저장할 데이터 준비
data = []

# 각 열의 데이터를 병합하여 저장
for idx, value in enumerate(first_column_values):
    row = [value.text.strip()]  # 첫 번째 열의 값 추가 (다섯 번째부터 시작)
    for col_values in second_and_more_columns_values:
        # 두 번째 열부터 71번째 열의 값은 처음부터 모두 추가
        row.append(col_values[idx] if idx < len(col_values) else "")
    data.append(row)

# CSV 파일로 저장
with open('output_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV 헤더 작성
    writer.writerow(column_names)
    
    # 데이터 작성
    writer.writerows(data)

print("Data successfully written to output_data.csv")

# 브라우저 닫기
driver.quit()


Data successfully written to output_data.csv
